<a href="https://colab.research.google.com/github/cobrien87/work/blob/main/MiniLesson1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is a Database?
Watch this video to find out:
[Youtube Video](https://www.youtube.com/watch?v=Tk1t3WKK-ZY)

The next 2 chunks of code will set up our environment and create functions in python to allow us to execute SQL Queries and turn csv files into SQL Tables.

*Instructions*  
*Step 1 - DOWNLOAD*:  Download the CSV files:  [Customers.csv](https://drive.google.com/file/d/1Jx1zB8IbFaBenLJ7efm9xx0IX4RpNb2c/view?usp=sharing)

*Step 2 - UPLOAD*:  Upload to the sample_data folder of Google Colab the file you just downloaded:  Customers.csv.   

In [ ]:
import pandas as pd
import sqlite3

def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table

    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()


def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe

    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.

    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

# Creating our tables!
We are using python pandas library to convert a csv file into a SQL table. 
We will skip the knitty gritty of creating tables in order to get to the more fun stuff. In real use-cases, SQL tables can be populated by already existing data or users entering in their information.

More on creating tables: [W3 Schools](https://www.w3schools.com/sql/sql_create_table.asp)

More on Insert(to populate the table): [W3 Schools](https://www.w3schools.com/sql/sql_insert.asp)

# Fields and Records
Tables contain rows and columns, where the rows are known as records and the columns are known as fields. A column is a set of data values of a particular type (like numbers or alphabets), one value for each row of the database, for example, Age, Student_ID, or Student_Name.

Here is a list of all SQL types: [W3 Schools](https://www.w3schools.com/sql/sql_datatypes.asp)

### The fields in our table Customers are cid, lname, fname, and zip

In [ ]:
#CUSTOMERS TABLE
# Step 1: Read the csv file into a dataframe
input_df = pd.read_csv('sample_data/Customers.csv')
 
# Step 2: Upload the dataframe to a SQL Table
pd_to_sqlDB(input_df,
            table_name='Customers',
            db_name='default.db')

2022-02-28 22:14:39 INFO: SQL DB default.db created


OperationalError: ignored

# SELECT
Selecting specific columns

```
SELECT column1, column2, ...
FROM table_name;
```

Selecting all columns from table


```
SELECT * FROM table_name;
```


More on Select Statements: [W3 Schools](https://www.w3schools.com/sql/sql_select.asp)

In [ ]:
# TODO: Get all columns from the Customers Table
sql_query_string = """
    SELECT * FROM Customers
"""
 
#Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,cid,lname,fname,zip
0,0,Berkowitz,Victoria,12345
1,1,Moore,Alex,67890
2,2,Kabanakis,Stephannia,11111
3,3,Zamansky,Mike,12345
4,4,DyrlandWeaver,JonAlf,67890
5,5,Wong,Tiffany,67890
6,6,Best,Michelle,11111
7,7,Scheffler,Ian,12345
8,8,Mueller,Brian,11111
9,9,Wilson,Eric,67890


In [ ]:
# TODO: Get the zip column from the Customers Table
sql_query_string = """
    SELECT zip FROM Customers
"""
 
#Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,zip
0,12345
1,67890
2,11111
3,12345
4,67890
5,67890
6,11111
7,12345
8,11111
9,67890


# ALIASING
SQL aliases are used to give a table, or a column in a table, a temporary name.

Aliases are often used to make column names more readable.

An alias only exists for the duration of that query.


Syntax for Aliasing:
```
SELECT column_name AS alias_name
FROM table_name;
```

More on Aliasing: [W3 Schools](https://www.w3schools.com/sql/sql_alias.asp)

In [ ]:
#ALIASING the Customers Table
sql_query_string = """
    SELECT fname AS FIRST, lname AS LAST FROM Customers
"""
 
#Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,FIRST,LAST
0,Victoria,Berkowitz
1,Alex,Moore
2,Stephannia,Kabanakis
3,Mike,Zamansky
4,JonAlf,DyrlandWeaver
5,Tiffany,Wong
6,Michelle,Best
7,Ian,Scheffler
8,Brian,Mueller
9,Eric,Wilson


In [ ]:
#ALIASING the Customers Table
#TODO: SELECT the fname AS FIRST, lname AS LAST, cid AS CustomerId from the Customers table
sql_query_string = """
    SELECT fname AS FIRST, lname AS LAST, cid AS CustomerID from Customers
"""
 
#Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,FIRST,LAST,CustomerID
0,Victoria,Berkowitz,0
1,Alex,Moore,1
2,Stephannia,Kabanakis,2
3,Mike,Zamansky,3
4,JonAlf,DyrlandWeaver,4
5,Tiffany,Wong,5
6,Michelle,Best,6
7,Ian,Scheffler,7
8,Brian,Mueller,8
9,Eric,Wilson,9


# WHERE CLAUSE
The WHERE clause is used to filter records.

It is used to extract only those records that fulfill a specified condition.

Syntax:
```
SELECT column1, column2, ...
FROM table_name
WHERE condition;
``` 

You can use AND and OR to filter to your needs
Syntax:
```
SELECT column1, column2, ...
FROM table_name
WHERE condition1 AND condition2;
``` 


### WHERE Clause Operators
* <, >, =, >=, <=
*   BETWEEN (Between a certain range)
*   LIKE (Search for a pattern)
*   IN (To specify multiple possible values for a column)

More on WHERE: [W3 Schools](https://www.w3schools.com/sql/sql_where.asp)

In [ ]:
#Selecting all rows/records from the Customers table where zipcode is 12345
sql_query_string = """
    SELECT * FROM Customers WHERE zip = 12345
"""
 
#Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,cid,lname,fname,zip
0,0,Berkowitz,Victoria,12345
1,3,Zamansky,Mike,12345
2,7,Scheffler,Ian,12345
3,11,Liu,Eric,12345
4,15,Wally,Mamadu,12345


In [ ]:
#TODO: Select all rows/records from the Customers table where zipcode is 11111
sql_query_string = """
    SELECT * FROM Customers WHERE zip = 11111 
"""
 
#Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

# LIKE
LIKE is used with the WHERE clause to find records that match a pattern

Syntax:
```
SELECT column1, column2, ...
FROM table_name
WHERE columnN LIKE pattern;
```

You can use ```%``` and ```_``` to find patterns

*   WHERE CustomerName LIKE 'a%'	
  *   Finds any values that start with "a"
*   WHERE CustomerName LIKE '%a'	
  *   Finds any values that end with "a"
*   WHERE CustomerName LIKE '%or%'	
  *   Finds any values that have "or" in any position
*   WHERE CustomerName LIKE '_r%'	
  *   Finds any values that have "r" in the second position
*   WHERE CustomerName LIKE 'a_%'	
  *   Finds any values that start with "a" and are at least 2 characters in length
*   WHERE CustomerName LIKE 'a__%'
   *   Finds any values that start with "a" and are at least 3 characters in length
*   WHERE ContactName LIKE 'a%o'	
   *   Finds any values that start with "a" and ends with "o"

More on LIKE [W3 Schools](https://www.w3schools.com/sql/sql_like.asp)

In [ ]:
#Selecting all rows/records from the Customers table where fname starts with E
sql_query_string = """
    SELECT * FROM Customers WHERE fname LIKE 'E%'
"""
 
#Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,cid,lname,fname,zip
0,9,Wilson,Eric,67890
1,10,Wingreen,Emma,67890
2,11,Liu,Eric,12345
3,12,Liete,Eduardo,11111


In [ ]:
#TODO: Select all rows/records from the Customers table where lname starts with W
sql_query_string = """
    SELECT * FROM Customers WHERE lname LIKE 'W%'
"""
 
#Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,cid,lname,fname,zip
0,5,Wong,Tiffany,67890
1,9,Wilson,Eric,67890
2,10,Wingreen,Emma,67890
3,15,Wally,Mamadu,12345


In [ ]:
#Selecting all rows/records from the Customers table where fname starts with B OR lname starts B 
sql_query_string = """
  YOUR QUERY HERE
"""
 
#Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df